## Preliminaries

In [ ]:
library(ape)
library(ggtree)
library(Quartet)
library(dplyr)

#### Function to display RF and Quartet metrics

In [ ]:
display_metrics <- function(tree, ref_tree) {
    splitStatuses <- SplitStatus(tree, ref_tree)
    quartetStatuses <- QuartetStatus(tree, ref_tree)
    print(paste("RF (normalized)", RawSymmetricDifference(splitStatuses, similarity = TRUE) / splitStatuses[, 'N']))
    print(paste("Quartet: ", SymmetricDifference(quartetStatuses, similarity = TRUE)))
}

## Comparison of trees

#### Loading trees

In [ ]:
## adjust paths accordingly
ref_tree <- read.tree("../config/esche_shige_short.tree")
rankc_tree <- read.tree("../rankc_nj_tree_1_renamed.nwk")
dcj_tree <- read.tree("../dcj_nj_tree_1_unimog.nwk")

#### Rankc vs. Reference

In [ ]:
display_metrics(rankc_tree, ref_tree)

#### Rank-Indel vs. Reference

In [ ]:
display_metrics(rankc_tree, ref_tree)

#### DCJ-Indel vs. Reference

In [ ]:
display_metrics(dcj_tree, ref_tree)

## Plotting

#### Reference

In [ ]:
png("reference_tree.png")
plot <- ggtree(ref_tree, branch.length = "none") +
    geom_tiplab() + 
    geom_hilight(node=37, fill="green") +
    geom_cladelabel(node=37, label="B1", color="green", align=TRUE, offset=3.5) +
    geom_hilight(node=32, fill="blue") +
    geom_cladelabel(node=32, label="A", color="blue", align=TRUE, offset=3.5) +
    geom_hilight(node=25, fill="red") +
    geom_cladelabel(node=25, label="B2", color="red", align=TRUE, offset=3.5) +
    geom_hilight(node=24, fill="orange") +
    geom_cladelabel(node=24, label="D", color="orange", align=TRUE, offset=3) +
    geom_hilight(node=9, fill="orange") +
    geom_cladelabel(node=9, label="D", color="orange", align=TRUE, offset=3.5) +
    geom_hilight(node=35, fill="purple") +
    geom_cladelabel(node=35, label="S", color="purple", align=TRUE, offset=3) +
    geom_hilight(node=16, fill="purple") +
    geom_cladelabel(node=16, label="S", color="purple", align=TRUE, offset=3.5) +
    geom_hilight(node=10, fill="purple") +
    geom_cladelabel(node=10, label="S", color="purple", align=TRUE, offset=3.5) +
    xlim(0, 20)
print(plot)
dev.off()

#### Rank

I removed the branch lenghts so that the plot works. Don't know why it breaks with branch lengths, even though I pass `branch.length="none"`.
```bash
cat rankc_nj_tree_1.nwk | sed 's/\:[0-9]*[\.[0-9]*]*//g' > rankc_nj_tree_1.nwk.nobranch
```

In [ ]:
rankc_tree_nobranch <- read.tree("../rankc_nj_tree_1_renamed.nwk.nobranch")

png("rankc_tree.png")
plot <- ggtree(rankc_tree_nobranch) +
    geom_tiplab() +
    geom_hilight(node=26, fill="green") +
    geom_cladelabel(node=26, label="B1", color="green", align=TRUE, offset=3.5) +
    geom_hilight(node=29, fill="blue") +
    geom_cladelabel(node=29, label="A", color="blue", align=TRUE, offset=3.5) +
    geom_hilight(node=31, fill="orange") +
    geom_cladelabel(node=31, label="D", color="orange", align=TRUE, offset=3.5) +
    geom_hilight(node=33, fill="red") +
    geom_cladelabel(node=33, label="B2", color="red", align=TRUE, offset=3.5) +
    geom_hilight(node=37, fill="purple") +
    geom_cladelabel(node=37, label="S", color="purple", align=TRUE, offset=3.5) +
    xlim(0, 20)
print(plot)
dev.off()

#### Rank-Indel

#### DCJ-indel

I removed the branch lenghts so that the plot works. Don't know why it breaks with branch lengths, even though I pass `branch.length="none"`.
```bash
cat dcj_nj_tree_1.nwk | sed 's/\:[0-9]*[\.[0-9]*]*//g' > dcj_nj_tree_1.nwk.nobranch
```

In [ ]:
dcj_tree_nobranch <- read.tree("../dcj_nj_tree_1_unimog.nwk.nobranch")

png("dcj_tree.png")
plot <- ggtree(dcj_tree_nobranch) +
    geom_tiplab() +
    geom_hilight(node=31, fill="green") +
    geom_cladelabel(node=31, label="B1", color="green", align=TRUE, offset=3.5) +
    geom_hilight(node=29, fill="blue") +
    geom_cladelabel(node=29, label="A", color="blue", align=TRUE, offset=3.5) +
    geom_hilight(node=34, fill="orange") +
    geom_cladelabel(node=34, label="D", color="orange", align=TRUE, offset=3) +
    geom_hilight(node=1, fill="orange") +
    geom_cladelabel(node=1, label="D", color="orange", align=TRUE, offset=3) +
    geom_hilight(node=35, fill="red") +
    geom_cladelabel(node=35, label="B2", color="red", align=TRUE, offset=3) +
    geom_hilight(node=39, fill="purple") +
    geom_cladelabel(node=39, label="S", color="purple", align=TRUE, offset=3) +
    geom_hilight(node=16, fill="purple") +
    geom_cladelabel(node=16, label="S", color="purple", align=TRUE, offset=3) +
    geom_hilight(node=19, fill="purple") +
    geom_cladelabel(node=19, label="S", color="purple", align=TRUE, offset=3) +
    xlim(0, 20)
print(plot)
dev.off()